In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

DEVICE=torch.device('cuda')

In [2]:
class Net(nn.Module):
    def __init__ (self,channel_conv,channel_nn,size):
        super(Net,self).__init__()
        
        self.block0=nn.Sequential(
            nn.Conv2d(in_channels=2, out_channels=channel_conv,kernel_size=3,padding=1),
            nn.BatchNorm2d(channel_conv), #channel number
            nn.LeakyReLU()
        )
        self.block1=nn.Sequential(
            nn.Conv2d(in_channels=channel_conv, out_channels=channel_conv,kernel_size=3,padding=1),
            nn.BatchNorm2d(channel_conv),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels=channel_conv, out_channels=channel_conv,kernel_size=3,padding=1),
            nn.BatchNorm2d(channel_conv)
        )
        self.fc0 = nn.Linear( 2*size*size ,channel_nn)
        self.fc1 = nn.Linear(channel_conv*size*size,channel_nn)
        self.fc2 = nn.Linear(channel_nn,channel_nn)
        self.fc3 = nn.Linear(channel_nn, 1)
        
        
    def forward(self,x):
        in_size = x.size(0)
  
        out = x.view(in_size,-1) # ( m, 2*size*size)
        out = self.fc0(out)     # ( 2*size*size to channel_nn)
        out1 = F.leaky_relu(out) # First transform Layer
        
        out =  self.fc2(out1)
        out = F.leaky_relu(out)
        out1 = F.leaky_relu(out1+ self.fc2(out))
        
        out =  self.fc2(out1)
        out = F.leaky_relu(out)
        out1 = F.leaky_relu(out1+ self.fc2(out))
        
        out =  self.fc2(out1)
        out = F.leaky_relu(out)
        out1 = F.leaky_relu(out1+ self.fc2(out))
        
        out =  self.fc2(out1)
        out = F.leaky_relu(out)
        out1 = F.leaky_relu(out1+ self.fc2(out))
        
        out =  self.fc2(out1)
        out = F.leaky_relu(out)
        out1 = F.leaky_relu(out1+ self.fc2(out))
        
        out =  self.fc2(out1)
        out = F.leaky_relu(out)
        out1 = F.leaky_relu(out1+ self.fc2(out))
        
        out =  self.fc2(out1)
        out = F.leaky_relu(out)
        out1 = F.leaky_relu(out1+ self.fc2(out))
        
        out =  self.fc2(out1)
        out = F.leaky_relu(out)
        out1 = F.leaky_relu(out1+ self.fc2(out))
        
        out =  self.fc2(out1)
        out = F.leaky_relu(out)
        out1 = F.leaky_relu(out1+ self.fc2(out))
        
        
        out = self.fc3(out1)
        
        return out
        

In [3]:
BATCH_SIZE=256
EPOCHS = 30
SIZE=8
CHANNEL_NN= 512
CHANNEL_CONV = 64

In [18]:
model = Net(channel_conv = CHANNEL_CONV,channel_nn = CHANNEL_NN,size = SIZE).to(DEVICE)

In [19]:
X1 = np.load(r'E:\modules\data\Covariance60_randomdb_-20~0_2.npy')
Y1 = np.load(r'E:\modules\data\lable60_randomdb_-20~0_2.npy')
criterion = nn.MSELoss()
X1_train,X1_test = X1[0:190000,:,:,:],X1[990000:1000000,:,:,:]
Y1_train,Y1_test = Y1[0:190000,:],Y1[990000:1000000,:]
x,x1 = torch.from_numpy(X1_train).float(),torch.from_numpy(X1_test).float()
y,y1 = torch.from_numpy(Y1_train).long(),torch.from_numpy(Y1_test).long()
train_data = torch.utils.data.TensorDataset(x,y)
test_data = torch.utils.data.TensorDataset(x1,y1)
train_loader = torch.utils.data.DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=True)

In [6]:
def train(model, device, train_loader, test_loader,optimizer, epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):  
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss1 = torch.sqrt(criterion(output,target.float()))
        loss1.backward()
        optimizer.step()
        if(batch_idx+1)%100 == 0: 
            model.eval()
            test_out=model(x1.to(device))
            loss2 = torch.sqrt( criterion(test_out,y1.float().to(device)))
            print('Train Epoch: {} [{}/{}] Train Loss: {:.6f} || Test Loss:{:.6f}'.format(
                epoch,(batch_idx+1) * len(data),len(train_loader.dataset),loss1.item(),loss2.item()))

In [26]:
BATCH_SIZE = 512
EPOCHS = 2
optimizer = optim.Adam(model.parameters(),lr=0.000003)
SIZE=8
CHANNEL_NN= 512
CHANNEL_CONV = 64

In [27]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE,train_loader,test_loader, optimizer, epoch)

Train Epoch: 1 [51200/190000] Train Loss: 3.113825 || Test Loss:3.357899
Train Epoch: 1 [102400/190000] Train Loss: 3.100776 || Test Loss:3.354762
Train Epoch: 1 [153600/190000] Train Loss: 3.542022 || Test Loss:3.354680
Train Epoch: 2 [51200/190000] Train Loss: 3.555429 || Test Loss:3.353069
Train Epoch: 2 [102400/190000] Train Loss: 3.068089 || Test Loss:3.355487
Train Epoch: 2 [153600/190000] Train Loss: 3.313504 || Test Loss:3.355757


In [71]:

torch.save(model,'E:\modules\my_model_ResDNN_60_random_recog_-20~0.pth')

In [ ]:
torch.cuda.empty_cache()

In [16]:
model.eval()
test_out=model(x1.to(DEVICE))
loss2 = torch.sqrt( criterion(test_out,y1.float().to(DEVICE)))
print(loss2)

tensor(1.4576, device='cuda:0', grad_fn=<SqrtBackward>)


In [138]:
y1[1].item()

-2

In [ ]:
model=torch.load('E:\modules\my_model4.pth').to(DEVICE)

In [7]:
torch.cuda_version

TypeError: 'NoneType' object is not callable